This base chatbot can have a medical conversation with a patient.

# Installation requirements

Run this Jupyther notebook in your favorite IDE, or here in this Google Collab environment.

If you run this locally, install the required libraries:
- pip install openai
- pip install ipywidgets


In [ ]:
# Import necessary libraries
from openai import OpenAI
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import HTML


# Configure OpenAI API
api_key = ''

client = OpenAI(api_key=api_key)


def initialize_chat():
    """
    Initialize the chat with a system message that sets the context for the medical chatbot.
    """
    return [{
        "role": "system",
        "content": """You are a helpful medical information assistant. You can provide general medical
        information and guidance, but you should always remind users that you're not a replacement
        for professional medical advice, diagnosis, or treatment."""
    }]


def chat_with_medical_bot():
    """
    Main function to run the medical chatbot interaction.
    """
    messages = initialize_chat()

    print("Medical Information Assistant: Hello! I can help you with general medical information.")
    print("Type 'quit' to end the conversation.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'quit':
            break

        # Add user message to conversation history
        messages.append({"role": "user", "content": user_input})

        # Get bot's response
        bot_response = get_chatbot_response(messages)

        # Add bot's response to conversation history
        messages.append({"role": "assistant", "content": bot_response})

        print(f"\nMedical Assistant: {bot_response}\n")



# Create widgets for the chat interface
output = widgets.Output()
text_input = widgets.Text(
    value='',
    placeholder='Type your medical question here...',
    description='',
    layout=widgets.Layout(width='80%')
)
send_button = widgets.Button(
    description='Send',
    button_style='primary',
    layout=widgets.Layout(width='19%')
)

# Create a container for chat history display
chat_history = widgets.HTML(
    value='<div style="height: 400px; overflow-y: auto; background-color: #f9f9f9; padding: 15px; border-radius: 5px;"></div>'
)

def update_chat_history(message, sender):
    """Update the chat history display with new messages"""
    current = chat_history.value[:-6]  # Remove closing div tag
    style = "background-color: #DCF8C6; margin: 5px; padding: 10px; border-radius: 10px;" if sender == "You" else "background-color: #E8E8E8; margin: 5px; padding: 10px; border-radius: 10px;"
    new_message = f'<div style="{style}"><strong>{sender}:</strong> {message}</div>'
    chat_history.value = current + new_message + "</div>"

def on_send_button_clicked(b):
    """Handle send button clicks"""
    user_message = text_input.value
    if user_message.strip() == '':
        return

    # Clear input
    text_input.value = ''

    # Update chat history with user message
    update_chat_history(user_message, "You")

    # Add user message to OpenAI messages
    messages.append({"role": "user", "content": user_message})

    # Get and display bot response
    bot_response = get_chatbot_response(messages)
    messages.append({"role": "assistant", "content": bot_response})
    update_chat_history(bot_response, "Medical Assistant")

# Connect button click to handler
send_button.on_click(on_send_button_clicked)

# Handle Enter key in text input
def on_enter(widget):
    if text_input.value.strip() != '':
        on_send_button_clicked(None)
text_input.on_submit(on_enter)

# Create main chat container
chat_container = widgets.VBox([
    widgets.HTML(value='<h2>Medical Information Assistant</h2>'),
    widgets.HTML(value='<p style="color: #666;">Ask me any medical questions, but remember I\'m not a replacement for professional medical advice.</p>'),
    chat_history,
    widgets.HBox([text_input, send_button])
])



In [ ]:
def get_chatbot_response(messages, max_completion_tokens=1500):
   """
    Get a response from the GPT model based on the conversation history.

    Args:
        messages (list): List of conversation messages
        max_completion_tokens (int): Maximum length of the response

    Returns:
        str: The chatbot's response
   """
   try:
       response = client.chat.completions.create(
           model="gpt-5-mini",
           messages=messages,
           max_completion_tokens=max_completion_tokens
       )
       return response.choices[0].message.content
   except Exception as e:
       return f"An error occurred: {str(e)}"

In [ ]:
# Initialize chat history
messages = initialize_chat()

# Display the chat interface
display(chat_container)